In [11]:
import os
import json

L_TOTAL = 32
L_RETAINED = 15
NUM_LAYER = L_TOTAL - L_RETAINED
LOCAL_DIR = f"/teamspace/studios/this_studio/nlp-project/outputs/Meta-Llama-3-8B/fused_{NUM_LAYER}_layers/iteration/merged_weights"

In [ ]:
!python pipeline.py \
    --ntrain 10 \
    --ngpu 2 \
    --model_path "meta-llama/Meta-Llama-3-8B" \
    --data_dir "/teamspace/studios/this_studio/nlp-project/data" \
    --num_layer {NUM_LAYER}


In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="meta-llama/Meta-Llama-3-8B",
    local_dir=LOCAL_DIR,
    allow_patterns=[
        "config.json",
        "original/tokenizer.model",
        "tokenizer_config.json",
        "special_tokens_map.json",
        "tokenizer.json",
        "generation_config.json"
    ]
)

In [ ]:
config_path = os.path.join(LOCAL_DIR, "config.json")

with open(config_path, "r") as f:
    config = json.load(f)

config["num_hidden_layers"] = L_RETAINED  

with open(config_path, "w") as f:
    json.dump(config, f, indent=4)

print("Updated num_hidden_layers =", config["num_hidden_layers"])

In [ ]:
!accelerate launch \
  -m lm_eval \
  --model hf \
  --model_args pretrained="/teamspace/studios/this_studio/nlp-project/outputs/Meta-Llama-3-8B/fused_17_layers/iteration/merged_weights",dtype=bfloat16 \
  --tasks mmlu \
  --batch_size 4 \
  --device cuda \
  --num_fewshot 5 \
  --output_path mmlu_eval_results.json